In [30]:
import torch
from dataLoader import VoicePassingDataloader
from model import VoicePassingModel
from transformers import DistilBertTokenizer, AdamW
from tqdm import tqdm

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [59]:
trainer_config = {
    'model' : VoicePassingModel(),
}

class VoicePassingTrainer():

    def __init__(self, model):

        self.device = "cuda" if torch.cuda.is_available() else 'cpu'
        self.model = model.to(self.device)
        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.train_loss_history = []
        self.train_acc_history = []
        self.valid_loss_history = []
        self.valid_acc_history = []

    def set_model(self, model):
        self.model = model.to(self.device)

    def get_history(self):

        history = {
            "train_loss" : self.train_loss_history,
            "train_accuracy" : self.train_acc_history,
            "valid_loss" : self.valid_loss_history,
            "valid_accuracy" : self.valid_acc_history
        }

        return history

    def train(self, num_epochs, train_loader, criterion, lr = 3e-5, valid_loader = None, reset_history = False, verbose = True):

        if reset_history:
            self.train_loss_history = []
            self.train_acc_history = []
            self.valid_loss_history = []
            self.valid_acc_history = []

        self.criterion = criterion
        self.optimizer = AdamW(params = self.model.parameters(), lr = lr, correct_bias=False)
        self.model.train()

        for epoch_idx in range(num_epochs):
            train_loss, train_acc = self.train_one_epoch(epoch_idx, train_loader, verbose = verbose)

            self.train_loss_history.append(train_loss)
            self.train_acc_history.append(train_acc)

            if valid_loader:
                valid_loss, valid_acc = self.validate(epoch_idx, valid_loader, verbose = verbose)

                # 모델 저장
                if not epoch_idx: # 저장된 게 없다면
                    torch.save(self.model.state_dict(), r"./result/best.pt")
                    print("the initial model saved")

                else:
                    if valid_loss < self.valid_loss_history[-1]:
                        torch.save(self.model.state_dict(), fr"./result/best.pt")
                        print("the best model saved")

                self.valid_loss_history.append(valid_loss)
                self.valid_acc_history.append(valid_acc)

        torch.save(self.model.state_dict(), fr"./result/last.pt")
        print("the last model saved")

    def train_one_epoch(self, index, train_loader, verbose = True):

        train_loss = 0
        train_correct = 0
        train_n_probs = 0

        for X, y in tqdm(train_loader, desc="batch", leave= True):

            X = self.tokenizer(
                text = X,
                add_special_tokens = True,
                max_length = 512,
                padding = "max_length",
                truncation = True,
                return_tensors = "pt"
            ).to(self.device)

            y = y.squeeze().to(self.device)

            pred = self.model(X)
        
            loss = self.criterion(pred, y)
            train_loss += loss.item()

            loss.backward()
            self.optimizer.step()

            pred_labels = pred.argmax(axis = 1)
            n_correct = len(torch.where(pred_labels == y)[0])

            train_correct += n_correct
            train_n_probs += len(y)

        train_acc = (train_correct / train_n_probs) * 100

        if verbose:
            print(f"EPOCH {index+1} TRAIN / Loss : {train_loss : .4f}, Acc : {train_acc : .4f}")
            print(pred[-4:])
            print(y[-4:])

        return train_loss, train_acc

    def test_a_sentence(self, text):

        X = self.tokenizer(
                text = text,
                add_special_tokens = True,
                max_length = 512,
                padding = "max_length",
                truncation = True,
                return_tensors = "pt"
            ).to(self.device)
        
        pred = self.model(X)
        return pred
    
    def validate(self, index, valid_loader, verbose = True):

        valid_correct = 0
        valid_n_probs = 0
        
        self.model.eval()

        for X, y in tqdm(valid_loader, desc="batch", leave= True):

            X = self.tokenizer(
                text = X,
                add_special_tokens = True,
                max_length = 512,
                padding = "max_length",
                truncation = True,
                return_tensors = "pt"
            ).to(self.device)

            y = y.squeeze().to(self.device)

            pred = self.model(X)        
            valid_loss = self.criterion(pred, y)

            pred_labels = pred.argmax(axis = 1)
            n_correct = len(torch.where(pred_labels == y)[0])

            valid_correct += n_correct
            valid_n_probs += len(y)

        valid_acc = (valid_correct / valid_n_probs) * 100

        if verbose:
            print(f"EPOCH {index+1} VALID / Loss : {valid_loss : .4f}, Acc : {valid_acc : .4f}")
            print(pred[-4:])
            print(y[-4:])
        
        return valid_loss, valid_acc
        

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
trainer = VoicePassingTrainer(model = VoicePassingModel())

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
criterion = torch.nn.CrossEntropyLoss()

In [62]:
trainer.train(10, VoicePassingDataloader(small = True), criterion = criterion, valid_loader=VoicePassingDataloader(test = True, small= True))

c:\Users\SSAFY\Desktop\S08P31A607\venv\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
batch: 100%|██████████| 4/4 [00:01<00:00,  2.20it/s]


EPOCH 1 TRAIN / Loss :  5.3532, Acc :  33.0000
tensor([[ 0.1320,  0.1297, -0.0846, -0.2382],
        [-0.0037,  0.1082, -0.1101, -0.2471],
        [-0.0243,  0.0892, -0.2200, -0.3167],
        [ 0.0628,  0.1843, -0.1001, -0.3303]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([1, 1, 1, 0], device='cuda:0')


batch: 100%|██████████| 4/4 [00:01<00:00,  2.36it/s]


EPOCH 1 VALID / Loss :  1.1504, Acc :  66.0000
tensor([[ 0.1725,  0.1283, -0.2461, -0.2035],
        [ 0.2789,  0.1716, -0.1987, -0.4033],
        [ 0.2379,  0.2562, -0.2289, -0.4677],
        [ 0.2129,  0.1559, -0.1788, -0.3683]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0, 0, 0, 0], device='cuda:0')
the initial model saved
the last model saved


batch: 100%|██████████| 4/4 [00:01<00:00,  2.32it/s]


...

EPOCH 9 VALID / Loss :  0.6963, Acc :  56.0000
tensor([[ 1.3091, -0.5882, -0.2857, -0.2399],
        [ 1.6467, -0.8838, -0.2116, -0.2572],
        [ 0.3119, -0.0555,  0.1473, -0.2136],
        [ 0.6371, -0.3572,  0.1195, -0.2275]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0, 0, 0, 0], device='cuda:0')
the last model saved


batch: 100%|██████████| 4/4 [00:01<00:00,  2.29it/s]


EPOCH 10 TRAIN / Loss :  4.5092, Acc :  52.0000
tensor([[-0.6079,  0.1439,  0.6449,  0.0373],
        [-0.4867,  0.0796,  0.5777,  0.0469],
        [-0.3991,  0.1092,  0.4638,  0.0647],
        [ 1.2057, -0.9383,  0.0224,  0.0389]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([1, 1, 1, 0], device='cuda:0')


batch: 100%|██████████| 4/4 [00:01<00:00,  2.30it/s]


EPOCH 10 VALID / Loss :  0.8101, Acc :  49.0000
tensor([[ 1.2925, -0.7846, -0.2641, -0.0117],
        [ 1.6141, -1.1644, -0.1879,  0.0967],
        [ 0.0769, -0.2408,  0.2363,  0.1720],
        [ 0.4971, -0.5475,  0.1816,  0.0898]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0, 0, 0, 0], device='cuda:0')
the last model saved
